###Introduction
CrewAI is a powerful framework that allows you to create, coordinate and deploy teams of AI agents that work together to accomplish complex tasks. This guide will walk you through the process of creating your own AI agent teams, even if you have minimal coding experience.

###install Required Libraries

In [1]:
!pip install -qU crewai langchain_openai 'crewai[tools]' google-search-results

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.0/633.0 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

###Getting Your API Keys
**Serper API Key**

* Visit https://serper.dev/api-key
* Sign up for an account or log in
* Navigate to the API dashboard
* Generate your API key
* Copy the key and use in your Colab Notebook

**Google Gemini API Key**

* Visit https://aistudio.google.com/
 Sign in with your Google account
* Navigate to "API Keys" in the settings menu
* Click "Create API Key"
* Copy your new API key And use in Colab Notebook


In [5]:
import os
from getpass import getpass

# Ask the user to enter the API keys (they won't show up on screen)
os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API key: ")
os.environ["SERPER_API_KEY"] = getpass("Enter your Serper API key: ")

Enter your Gemini API key: ··········
Enter your Serper API key: ··········


###Initialize Models

In [7]:
from langchain_openai import ChatOpenAI
from crewai import LLM

Gemini = LLM(
    model="gemini/gemini-2.0-flash",)


#gpt4o = ChatOpenAI(model = "gpt-4o")

### AI Customer Support Agent 🧳


In [8]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI
from crewai import LLM

# Initialize tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
tools = [search_tool, scrape_tool]

def create_customer_support_system(customer_query, product_name, knowledge_base_url):
    """
    Creates and runs a customer support system with multiple specialized agents
    to handle customer inquiries.

    Args:
        customer_query (str): The customer's support request or question
        product_name (str): Name of the product customer is inquiring about
        knowledge_base_url (str): URL to the product's knowledge base/documentation

    Returns:
        str: A comprehensive support response
    """
    # Define the Query Classifier Agent
    classifier_agent = Agent(
        role="Support Query Classifier",
        goal="Accurately classify customer support queries by type and priority",
        backstory="An expert in customer support triage with the ability to identify query type, urgency, and required expertise.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Technical Support Agent
    technical_agent = Agent(
        role="Technical Support Specialist",
        goal=f"Provide accurate technical solutions for {product_name} issues",
        backstory=f"A senior engineer with deep knowledge of {product_name}'s architecture and common technical issues. Known for explaining complex solutions in simple terms.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Customer Relations Agent
    customer_agent = Agent(
        role="Customer Relations Manager",
        goal="Ensure customer satisfaction and provide empathetic communication",
        backstory="A customer service expert specialized in creating positive customer experiences, with expertise in communication psychology and conflict resolution.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Tasks

    # Task 1: Classify the query
    classification_task = Task(
        description=f"Analyze this customer query: '{customer_query}' about product '{product_name}'. " +
                   "Classify it by type (technical issue, billing question, feature request, etc.), " +
                   "determine priority level (low, medium, high, urgent), and identify what expertise is needed to resolve it.",
        expected_output="A detailed classification with query type, priority level, required expertise, and recommended next steps.",
        agent=classifier_agent
    )

    # Task 2: Technical solution research
    technical_task = Task(
        description=f"Research and provide a solution for this query: '{customer_query}' about '{product_name}'. " +
                   f"Use the knowledge base at {knowledge_base_url} and search for relevant documentation. " +
                   "Provide step-by-step troubleshooting instructions with screenshots or diagrams if helpful.",
        expected_output="A comprehensive technical solution with step-by-step instructions that completely resolves the customer's issue.",
        agent=technical_agent
    )

    # Task 3: Craft the final response
    response_task = Task(
        description=f"Using the technical solution provided, craft a friendly, empathetic response to '{customer_query}'. " +
                  "The response should be professional but warm, easy to understand, and should anticipate follow-up questions. " +
                  "Include appropriate reassurances and clear next steps if the issue requires additional support.",
        expected_output="A complete customer-ready response that balances technical accuracy with customer care principles.",
        agent=customer_agent
    )

    # Create and Run the Crew
    support_crew = Crew(
        agents=[classifier_agent, technical_agent, customer_agent],
        tasks=[classification_task, technical_task, response_task],
        verbose=True,
        process=Process.sequential
    )

    result = support_crew.kickoff(
        inputs={
            "customer_query": customer_query,
            "product_name": product_name,
            "knowledge_base_url": knowledge_base_url
        }
    )

    return result

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


### Execute the agent


In [9]:
print("AI Customer Support System")
print("==========================")

product_name = input("Enter product name: ")
knowledge_base_url = input("Enter knowledge base URL: ")
print("\nWhat issue is the customer experiencing?")
customer_query = input("> ")

print("\nProcessing support request...\n")
support_response = create_customer_support_system(customer_query, product_name, knowledge_base_url)

print("\nCustomer Support Response:")
print("==========================")
print(support_response)

AI Customer Support System
Enter product name: bottle 
Enter knowledge base URL: nth

What issue is the customer experiencing?
> it is not of good quality 

Processing support request...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ea8f97d7-ca61-4dca-949a-f7c85ae4b2c8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Query Classifier                                                                                │
│                                                                                                                 │
│  Task: Analyze this customer query: 'it is not of good quality ' about product 'bottle '. Classify it by type   │
│  (technical issue, billing question, feature request, etc.), determine priority level (low, medium, high,       │
│  urgent), and identify what expertise is needed to resolve it.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Relations Manager                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  We're so sorry to hear you're experiencing quality issues with your bottle. We understand how frustrating      │
│  that can be.                                                                                                   │
│                                                                                                                 │
│  To help us understand the problem better and find the best solution for you, could you please provide a bit    │
│  more detail about what you're seeing? For example:                                                             │
│                                                                                                                 │
│  *   Is the bottle made of glass or plastic (and if plastic, do you know what type)?                            │
│  *   Could you describe the specific issue you're experiencing (e.g., leaking, cracks, discoloration,           │
│  difficulty sealing, etc.)?                                                                                     │
│  *   If possible, could you send us a photo of the defect?                                                      │
│  *   What will you be using the bottle for?                                                                     │
│                                                                                                                 │
│  In the meantime, here are a few general things to check:                                                       │
│                                                                                                                 │
│  *   **For all bottles:** Check for any cracks, chips, or holes. Make sure the cap or closure creates a tight   │
│  seal.                                                                                                          │
│  *   **For glass bottles:** Look for bubbles (blisters), wavy lines (cords), or foreign material embedded in    │
│  the glass (inclusions).                                                                                        │
│  *   **For plastic bottles:** Check if the bottle neck is even, if the cap threads align correctly, and if      │
│  there are any excessively raised lines (mold seams). Also, gently squeeze the bottle to check for thin spots.  │
│                                                                                                                 │
│  Based on what you describe, we'll be able to determine if a replacement or refund is the right course of       │
│  action. We appreciate your patience and look forward to hearing from you!                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 872d13c2-a458-42d9-aaa8-3e34974eb1ea                                                                     │
│  Agent: Customer Relations Manager                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ea8f97d7-ca61-4dca-949a-f7c85ae4b2c8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: We're so sorry to hear you're experiencing quality issues with your bottle. We understand how    │
│  frustrating that can be.                                                                                       │
│                                                                                                                 │
│  To help us understand the problem better and find the best solution for you, could you please provide a bit    │
│  more detail about what you're seeing? For example:                                                             │
│                                                                                                                 │
│  *   Is the bottle made of glass or plastic (and if plastic, do you know what type)?                            │
│  *   Could you describe the specific issue you're experiencing (e.g., leaking, cracks, discoloration,           │
│  difficulty sealing, etc.)?                                                                                     │
│  *   If possible, could you send us a photo of the defect?                                                      │
│  *   What will you be using the bottle for?                                                                     │
│                                                                                                                 │
│  In the meantime, here are a few general things to check:                                                       │
│                                                                                                                 │
│  *   **For all bottles:** Check for any cracks, chips, or holes. Make sure the cap or closure creates a tight   │
│  seal.                                                                                                          │
│  *   **For glass bottles:** Look for bubbles (blisters), wavy lines (cords), or foreign material embedded in    │
│  the glass (inclusions).                                                                                        │
│  *   **For plastic bottles:** Check if the bottle neck is even, if the cap threads align correctly, and if      │
│  there are any excessively raised lines (mold seams). Also, gently squeeze the bottle to check for thin spots.  │
│                                                                                                                 │
│  Based on what you describe, we'll be able to determine if a replacement or refund is the right course of       │
│  action. We appreciate your patience and look forward to hearing from you!                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Customer Support Response:
We're so sorry to hear you're experiencing quality issues with your bottle. We understand how frustrating that can be.

To help us understand the problem better and find the best solution for you, could you please provide a bit more detail about what you're seeing? For example:

*   Is the bottle made of glass or plastic (and if plastic, do you know what type)?
*   Could you describe the specific issue you're experiencing (e.g., leaking, cracks, discoloration, difficulty sealing, etc.)?
*   If possible, could you send us a photo of the defect?
*   What will you be using the bottle for?

In the meantime, here are a few general things to check:

*   **For all bottles:** Check for any cracks, chips, or holes. Make sure the cap or closure creates a tight seal.
*   **For glass bottles:** Look for bubbles (blisters), wavy lines (cords), or foreign material embedded in the glass (inclusions).
*   **For plastic bottles:** Check if the bottle neck is even, if the cap

Absolutely! You're building a **multi-agent AI-powered customer support system** using **CrewAI**, **LangChain**, and **Google Gemini**. Let's walk through the code step-by-step in plain English.

---

## 🧠 High-Level Overview:

You're using **three AI agents**, each with their own role:

1. **Classifier Agent** → Understands the type and urgency of the query.
2. **Technical Agent** → Finds the actual solution using online tools.
3. **Customer Agent** → Writes a friendly, complete customer response.

---

## 🧱 Libraries and Setup

```python
from langchain_openai import ChatOpenAI
from crewai import LLM
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
```

* `crewai`: Lets you define and run a group (crew) of cooperating agents.
* `LLM`: Initializes the large language model (you used Gemini).
* `SerperDevTool`: Tool to search the web (via Google Search API).
* `ScrapeWebsiteTool`: Tool to scrape content from URLs like a knowledge base.

```python
Gemini = LLM(model="gemini/gemini-2.0-flash")
```

* You specify **Gemini 2.0 Flash** as the language model each agent will use.

---

## 🔧 Initialize Tools

```python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
tools = [search_tool, scrape_tool]
```

These tools are passed to agents so they can **search the web** or **scrape documentation** when solving problems.

---

## 🧩 `create_customer_support_system()` Function

This is the core function you call. You pass:

* `customer_query`: What the customer is asking.
* `product_name`: What product they’re asking about.
* `knowledge_base_url`: A website where answers may be found.

It returns a **support response** generated by the agents working together.

---

## 👷 3 Specialized Agents

Each agent has:

* A **role** (who they are)
* A **goal** (what they’re trying to achieve)
* A **backstory** (to help the LLM behave with character)
* The Gemini LLM
* Access to web tools

### 1. `classifier_agent`

* Classifies the query (type: tech/billing/etc., urgency level, etc.)

### 2. `technical_agent`

* Solves the actual technical problem by researching the knowledge base.

### 3. `customer_agent`

* Writes a friendly, professional reply using the technical solution.

---

## ✅ Tasks for Each Agent

Each task:

* Has a `description` telling the agent what to do
* Specifies an `expected_output`
* Assigns an `agent` to do the work

```python
classification_task → classifier_agent
technical_task → technical_agent
response_task → customer_agent
```

---

## 🤝 Crew = Agents + Tasks

```python
support_crew = Crew(
    agents=[classifier_agent, technical_agent, customer_agent],
    tasks=[classification_task, technical_task, response_task],
    verbose=True,
    process=Process.sequential  # Tasks run one after the other
)
```

The **crew** runs tasks **in order**, using the outputs from earlier tasks in later ones.

---

## ▶️ Run the System

```python
result = support_crew.kickoff(
    inputs={
        "customer_query": customer_query,
        "product_name": product_name,
        "knowledge_base_url": knowledge_base_url
    }
)
```

* This kicks off the process and lets the agents work together.
* Each agent takes input, uses its tools + Gemini LLM, and produces its part.
* The final result is the **polished customer response**.

---

## 🧪 User Interface (Console I/O)

```python
product_name = input("Enter product name: ")
knowledge_base_url = input("Enter knowledge base URL: ")
customer_query = input("What issue is the customer experiencing? ")
```

Then it prints the final customer-facing reply that combines:

* Classification
* Technical fix
* Friendly explanation

---

## ✅ Summary

| Component            | What it does                                  |
| -------------------- | --------------------------------------------- |
| `Agent`              | AI worker with a role, tools, and personality |
| `Task`               | Specific job for an agent                     |
| `Crew`               | Team of agents + task workflow                |
| `Gemini`             | The LLM that powers each agent                |
| `SerperDevTool`      | Lets agents Google search                     |
| `ScrapeWebsiteTool`  | Lets agents extract info from URLs            |
| `Process.sequential` | Ensures tasks run in order                    |

---

## Want a Real Example?

I can give you a test case to run — like a customer asking:

> "My app keeps crashing when I try to upload a photo. Can you help?"

Would you like that?
